# Bibliotecas

In [1]:
from utils.auxiliares import ScraperZap
from utils.bases_psql import UtilsPSQL
import psycopg2
import os
import pandas as pd
import time


# Criando Base de Dados

In [9]:
# Checando tabelas disponíveis

UtilsPSQL().tables_info()

,tabela,colunas
0,dados_geograficos,8
1,requests_municipios,6
2,scraping_imoveis,20


In [11]:
# Criando base dados, no banco 'scraping' do postgre, para receber os dados do web scraping

dict_colunas = {
        'transacao':'VARCHAR',
        'base':'VARCHAR',
        'local':'VARCHAR',
        'tipo':'VARCHAR',
        'subtipo':'VARCHAR',
        'id':'VARCHAR',
        'url':'VARCHAR',
        'destaque':'VARCHAR',
        'bairro':'VARCHAR',
        'endereco':'VARCHAR',
        'descricao':'VARCHAR',
        'area': 'DOUBLE PRECISION',
        'quartos':'DOUBLE PRECISION',
        'chuveiros':'DOUBLE PRECISION',
        'garagens':'DOUBLE PRECISION',
        'aluguel':'DOUBLE PRECISION',
        'total':'DOUBLE PRECISION',
        'valor_abaixo':'VARCHAR',
        'data':'VARCHAR',
        'mes':'VARCHAR'
}

UtilsPSQL().create_table(table_name = 'dados_imoveis_raw', table_columns=dict_colunas)

Tabela 'dados_imoveis_raw' foi criada com sucesso!


In [12]:
# Checando

UtilsPSQL().tables_info()

,tabela,colunas
0,dados_geograficos,8
1,dados_imoveis_raw,20
2,requests_municipios,6
3,scraping_imoveis,20


# Web Scraping

### Auxiliares

In [14]:
# Carregando dados geográficos

base_mun = UtilsPSQL().columns_info(table_name = 'requests_municipios', modo = 'full')

base_mun.head()

,id_municipio,str_municipio,str_uf,status_code,imoveis,dt_data
0,1200013,Acrelândia,AC,404.0,0,2023-08-20
1,1200054,Assis Brasil,AC,404.0,0,2023-08-20
2,1200104,Brasiléia,AC,404.0,0,2023-08-20
3,1200138,Bujari,AC,404.0,0,2023-08-20
4,1200179,Capixaba,AC,404.0,0,2023-08-20


### Scraping de todos os imóveis, especificando o tipo

In [ ]:
# Iterando em cada página, obtendo os dados de cada card de imóvel, salvando numa lista e passando para a próxima página

# Lista de imóveis que será usada para criar o dataframe após finalizar cada localidade
lista_imoveis = []

# Lista das categorias (residencial ou comercial) de imóveis disponíveis
categoria_imovel = [imoveis_residenciais, imoveis_comerciais]

# Controle para especificar o que será buscado, visto que cidades com grande número de imóveis aparecem com resultado limitado, já que o número máximo de páginas da zapimoveis é 100. Então, é possível rodar mais de uma vez para algumas cidades.
tipo_loop = input('Tipo de Loop: 1. UF | 2.Local | 3. Região | 4. Municípios e Iniciais | 5. Completo')

if tipo_loop == '1' or tipo_loop.upper() == 'UF':
    uf = list(re.sub("[\s]","",input("Estados a serem rodados: ").upper()).split(','))
    df_atual = base_mun[base_mun['str_uf'].isin(uf)].reset_index(drop = True)
    print(f'Scraping será rodado para os seguintes Estados: {uf}')

elif tipo_loop == '2' or tipo_loop.upper() == 'LOCAL':
    mun = list(re.sub("[\s]","",input("Locais a serem rodados: ").lower()).split(','))
    df_atual = base_mun[base_mun['str_local'].isin(mun)].reset_index(drop = True)
    print(f'Scraping será rodado para os seguintes locais: {mun}')

elif tipo_loop == '3' or tipo_loop.lower() == 'região':
    reg = list(re.sub("[\s]","",input("Regiões a serem rodadas: ")).split(','))
    df_atual = base_mun[base_mun['str_regiao'].isin(reg)].reset_index(drop = True)
    print(f'Scraping será rodado para as seguintes Regiões: {reg}')

elif tipo_loop == '4' or tipo_loop.title() == 'Municípios e Iniciais':
    uf2 = list(re.sub("[\s]","",input("Estados a serem rodados: ").upper()).split(','))
    iniciais = tuple(re.sub("[\s]","",input("Iniciais dos municípios a serem rodados: ").upper()).split(','))
    df_atual = base_mun.loc[(base_mun['str_municipio'].str.startswith(iniciais)) & (base_mun['str_uf'].isin(uf2))].reset_index(drop = True)
    print(f'Scraping será rodado para os estados {uf2} com os municípios que iniciem com as seguintes letras: {iniciais}')

else:
    df_atual = base_mun
    print('Scraping será rodado para todos os municípios!')

# Scraping
# Loop que define a linha atual do dataframe com os locais de busca
for linha in range(0,df_atual.shape[0]):
    
    # Definindo local de busca com base na linha atual do dataframe
    local = df_atual.at[linha,'str_local']
    
    # Definindo a categoria que será buscada
    for categoria in categoria_imovel:

        # Definindo os tipos de imóveis que serao buscados dentro da categoria e local atual
        for tipo, subtipos in categoria.items():
    
            # Iterando nas páginas disponíveis do local atual
            for pagina in range(1,200):

                # Definindo User Agents
                ua = UserAgent()
                user_agents = ua.get_random_user_agent()

                # Dados da Requisição
                #url = f'https://www.zapimoveis.com.br/aluguel/imoveis/{local}/?pagina={pagina}&tipo=Im%C3%B3vel%20usado&transacao=Aluguel'
                # url_ex = f'https://www.zapimoveis.com.br/aluguel/{tipo}/{local}/?&transacao=Aluguel&tipo=Im%C3%B3vel%20usado&tipos={subtipos}&pagina={pagina}'
                url_ex = f'https://www.zapimoveis.com.br/aluguel/{tipo}/se+aracaju/?&transacao=aluguel&&tipos={subtipos}&pagina={pagina}'
                headers = {'user-agent': user_agents.strip(), 'encoding':'utf-8'}
                
                # Encoding
                url = url_ex.encode('utf-8')

                #try: Usando esse try-except, muitas locais nao estavam retornando todos os imóveis. Necessário rever formas de reverter o UnicodeDecodeError
                #   url = url_ex.encode('utf-8')
                #   requisicao = requests.get(url, headers = headers)
                #except UnicodeDecodeError:
                #    url = urlencode(url_ex)
                #    requisicao = requests.get(url, headers = headers)
                #except: 
                #   url = str(url_ex).encode('unicode_escape')
                #   requisicao = requests.get(url, headers = headers) 

                # Loop de tentativas em caso de erro 
                tentativas = 10

                for i in range(1,tentativas):

                    # Testando a requisicao
                    try:
                        requisicao = requests.get(url, headers = headers)
                        break

                    # Retornando erro de conexao
                    except requests.exceptions.ConnectionError as e:
                        print(f"Erro de Conexão: {e}")
                        print(f"Tentando Reconectar em {time.sleep(int(tentativas)*2)} segundos...")

                    # Retornando erro de encoding
                    except UnicodeDecodeError as u:
                        print(f'Erro de Encoding: {u}')
                        url = str(url_ex).encode('unicode_escape')
                        requisicao = requests.get(url, headers = headers)
                        break

                # Requisição
                requisicao = requests.get(url, headers = headers)

                # Criando sessão e definindo reconexão em caso do erro de conexão
                # session = requests.Session()
                # retry = Retry(connect = 10, backoff_factor = 0.5)
                # adapter = HTTPAdapter(max_retries = retry)
                # session.mount('http://', adapter)
                # session.mount('https://', adapter)
                # session.get(url)

                # Verifica se a página, apesar de retornar status_code = 200, não possui imóveis
                sel = Selector(requisicao)
                card_sem_imoveis = sel.xpath('//div[@class="no-results__container"]//text()').extract()

                # Iterar apenas se a página tiver status_code 200 e não possuir a tag que indica página sem imóveis
                if requisicao.status_code == 200 and len(card_sem_imoveis) == 0:

                    # Tentar obter dados da página
                    tentativa = 0
                
                    while tentativa < 10:
                        
                        # Tentativa de obter dados de imóveis da página atual
                        try:
                            # Retorna todos os cards com imóveis da página
                            soup = BeautifulSoup(requisicao.text, 'html.parser')
                            cards = soup.find('div',{'class':'listings__container'}).find_all('div',class_='card-container js-listing-card')

                            # Obtém os ids de todos os cards usando Scrapy
                            sel = Selector(requisicao)
                            card_ids = sel.xpath('//div[@class="listings__container"]//@data-id').extract()

                            # Itera sobre todos os cards da página e faz o append desses dados na lista_imoveis, posteriormente passa para a próxima página
                            for card in cards:
                                # Base de busca
                                base = 'zapimoveis'

                                # Cidade do imóvel
                                cidade_imo = df_atual.at[linha,'str_municipio']

                                # UF do imóvel
                                uf_imovel = df_atual.at[linha,'str_uf']

                                # ID do município
                                id_mun = df_atual.at[linha,'id_municipio']

                                # ID do card, mediante Selector da lib scrapy, usando o atributo @ e o método extract para retornar todos os id do campo data-id
                                id_card = card_ids[cards.index(card)]

                                # Categoria do imóvel
                                if categoria_imovel.index(categoria) == 0:
                                    categoria_imo = 'Residencial'
                                else:
                                    categoria_imo = 'Comercial'
                                    
                                # Tipo do imóvel
                                tipo_imo = tipo

                                # Subtipo do imóvel
                                subtipo_imo = subtipos

                                # Se possui destaque
                                try:
                                    destaque = card.find('small',{'class':'color-white text-small text-small__bolder label__container background-color-highlight-1'}).text
                                except:
                                    destaque = 'Não Possui Destaque'

                                # Aluguel
                                try:
                                    aluguel = float(((re.sub("[^0-9,\\.]",'',card.find('div',{'class':'oz-datazap-stamp oz-datazap-stamp--dynamic'}).text)).replace('.','')).replace(',','.'))
                                except:
                                    aluguel = None

                                # Condomínio
                                try:
                                    #condominio = card.find('li',{'class':'card-price__item condominium text-regular'}).find('span',{'class':'card-price__value'}).text
                                    condominio = float(((re.sub("[^0-9,\\.]",'',card.find('li',{'class':'card-price__item condominium text-regular'}).find('span',{'class':'card-price__value'}).text)).replace('.','')).replace(',','.'))
                                except:
                                    condominio = None

                                # IPTU
                                try:
                                    #iptu = card.find('li',{'class':'card-price__item iptu text-regular'}).find('span',{'class':'card-price__value'}).text
                                    iptu = float(((re.sub("[^0-9,\\.]",'',card.find('li',{'class':'card-price__item iptu text-regular'}).find('span',{'class':'card-price__value'}).text)).replace('.','')).replace(',','.'))
                                except:
                                    iptu = None

                                # Descrição
                                try:
                                    descricao = card.find('div',{'class':'collapse simple-card__description'}).find('span',{'class':'simple-card__text text-regular'}).text.replace('\n','').lower()
                                except:
                                    descricao = None

                                # Endereço
                                try:
                                    endereco = card.find('h2',{'class':'simple-card__address color-dark text-regular'}).text.replace('\n','').lower()
                                except:
                                    endereco = None

                                # Área
                                try:
                                    #area = card.find('ul',{'class':'feature__container simple-card__amenities'}).find('span',{'itemprop':'floorSize'}).text
                                    area = float(((re.sub("[^0-9,\\.]",'',card.find('ul',{'class':'feature__container simple-card__amenities'}).find('span',{'itemprop':'floorSize'}).text)).replace('.','')).replace(',','.'))
                                except:
                                    area = None

                                # Quartos
                                try:
                                    #quartos = card.find('ul',{'class':'feature__container simple-card__amenities'}).find('span',{'itemprop':'numberOfRooms'}).text
                                    quartos = float(((re.sub("[^0-9,\\.]",'',card.find('ul',{'class':'feature__container simple-card__amenities'}).find('span',{'itemprop':'numberOfRooms'}).text)).replace('.','')).replace(',','.'))
                                except:
                                    quartos = None

                                # Garagens
                                try:
                                    #garagens = card.find('ul',{'class':'feature__container simple-card__amenities'}).find('li',{'class':'feature__item text-small js-parking-spaces'}).text.replace('parking','').strip()
                                    garagens = float(((re.sub("[^0-9,\\.]",'',card.find('ul',{'class':'feature__container simple-card__amenities'}).find('li',{'class':'feature__item text-small js-parking-spaces'}).text.replace('parking','').strip())).replace('.','')).replace(',','.'))
                                except:
                                    garagens = None

                                # Banheiros
                                try:
                                    #banheiros = card.find('ul',{'class':'feature__container simple-card__amenities'}).find('span',{'itemprop':'numberOfBathroomsTotal'}).text
                                    banheiros = float(((re.sub("[^0-9,\\.]",'',card.find('ul',{'class':'feature__container simple-card__amenities'}).find('span',{'itemprop':'numberOfBathroomsTotal'}).text)).replace('.','')).replace(',','.'))
                                except:
                                    banheiros = None

                                # Data completa de Extração
                                data = datetime.datetime.now(tz = pytz.timezone('America/Sao_Paulo')).strftime("%d/%m/%Y")

                                # Mês de Extração
                                mes = datetime.datetime.now(tz = pytz.timezone('America/Sao_Paulo')).strftime("%m/%Y")

                                # Incluindo os valores da página na lista
                                lista_imoveis.append(
                                    list(
                                        [
                                            base,
                                            cidade_imo,
                                            uf_imovel,
                                            id_mun,
                                            id_card,
                                            categoria_imo,
                                            tipo_imo,
                                            subtipo_imo,
                                            destaque,
                                            aluguel,
                                            condominio,
                                            iptu,
                                            descricao,
                                            endereco,
                                            area,
                                            quartos,
                                            garagens,
                                            banheiros,
                                            data,
                                            mes
                                        ]
                                    )
                                )

                            # Relatório do total de imóveis por página
                            hora = datetime.datetime.now(tz = pytz.timezone('America/Sao_Paulo')).strftime("%d/%m/%Y %H:%M:%S")
                            print(f'\n{hora} - Local {local}, tipo {tipo}, Página {pagina}, {len(cards)} imóveis')

                            # Parando as tentativas em caso de a página retornar dados
                            break
                        # Em caso de erro na tentativa de obter os dados da página, tentar mais uma vez até o número máximo de 10, porque o erro pode ser diverso, como oscilacao da internet, etc
                        except:
                            tentativa += 1
                            hora = datetime.datetime.now(tz = pytz.timezone('America/Sao_Paulo')).strftime("%d/%m/%Y %H:%M:%S")
                            print(f'{hora} - Página {pagina} do local {local} - Tentativa {tentativa}')
                            time.sleep(int(tentativa))

                elif requisicao.status_code == 429:
                    # Erro de muitas requisições
                    time.sleep(60)
                    continue
                elif (requisicao.status_code != 200 or len(card_sem_imoveis) > 0) and requisicao != None:
                    hora = datetime.datetime.now(tz = pytz.timezone('America/Sao_Paulo')).strftime("%d/%m/%Y %H:%M:%S")
                    #print(f'\n{hora} - Local {local}, tipo {tipo}, concluído! {pagina-1} páginas de imóveis retornadas, até {(pagina-1)*100} imóveis.')
                    print(f'\n{hora} - Local {local}, tipo {tipo}, concluído! {pagina-1} páginas de imóveis retornadas. Status {requisicao.status_code}')
                    print('-'*140)
                    #time.sleep(1)
                    break
                else:
                    continue


        # Criando o dataframe de repasse
        imoveis_df_repasse = pd.DataFrame(
                    lista_imoveis,
                    columns = [
                        "base_busca",
                        "str_municipio",
                        "str_estado",
                        "id_municipio",
                        "id_imo",
                        "str_categoria_imo",
                        "str_tipo_imo",
                        "str_subtipo_imo",
                        "str_destaque",
                        "vl_luguel",
                        "vl_condominio",
                        "vl_iptu",
                        "str_descricao",
                        "str_endereco",
                        "vl_area",
                        "vl_quartos",
                        "vl_garagens",
                        "vl_banheiros",
                        "dt_data_extracao",
                        "dt_mes_extracao"
                    ]
        )

    # Incluindo o Resultado de cada loop em uma localidade específica
    #imoveis_df = pd.concat([imoveis_df,imoveis_df_repasse])
    imoveis_df = pd.concat([imoveis_df, imoveis_df_repasse], ignore_index = True)

    # Removendo imóveis duplicados com o mesmo mês de extração
    imoveis_df.drop_duplicates(subset = ["id_imo","str_municipio","dt_mes_extracao"], inplace = True)
    imoveis_df.reset_index(drop = True)

    # Vendo estados já carregados no dataframe
    estados = list(imoveis_df['str_estado'].unique())
    print(f'\nEstados já carregados no dataframe: {estados}')